In [67]:
import requests
import os
import json
from pprint import pprint
import warnings

In [68]:
ZENODO_ACCESS_TOKEN = os.environ.get('ZENODO_ACCESS_TOKEN', '4WUTJLhMlKljZkknoc7llyMcTy3vjBy3I11j0mwUHVSYixxjt9Q72kLo8KMp')
ZENODO_URL = os.environ.get('ZENODO_URL', 'https://sandbox.zenodo.org')
params = {'access_token': ZENODO_ACCESS_TOKEN}
headers = {"Content-Type": "application/json"}

In [69]:
publish_path='publish/1.11'
#Read metadata from zenodo.json file
with open(f'{publish_path}/zenodo.json', 'r') as f:
  zenodo_json = json.load(f)
zenodo_json

{'metadata': {'title': 'NetCDF Climate and Forecast (CF) Metadata Conventions',
  'description': 'This document describes the CF conventions for climate and forecast metadata designed to promote the processing and sharing of files created with the netCDF Application Programmer Interface. The conventions define metadata that provide a definitive description of what the data in each variable represents, and of the spatial and temporal properties of the data. This enables users of data from different sources to decide which quantities are comparable, and facilitates building applications with powerful extraction, regridding, and display capabilities. The CF conventions generalize and extend the COARDS conventions. The extensions include metadata that provides a precise definition of each variable via specification of a standard name, describes the vertical locations corresponding to dimensionless vertical coordinate values, and provides the spatial coordinates of non-rectilinear gridded d

In [70]:
# Create empty record
res = requests.post(
  f'{ZENODO_URL}/api/deposit/depositions',
  params=params,
  json={},
  headers=headers
)

assert (res.status_code == 201), f'Error creating record: {res.json()}'

newrecord = res.json()
newrecord_deposition_id = newrecord['id']
newrecord_bucket_url = newrecord["links"]["bucket"]
newversion_prereserve_doi = newrecord['metadata']['prereserve_doi']['doi']
pprint(newrecord)

{'conceptrecid': '130528',
 'created': '2024-11-13T12:19:32.357191+00:00',
 'files': [],
 'id': 130529,
 'links': {'badge': 'https://sandbox.zenodo.org/badge/doi/.svg',
           'bucket': 'https://sandbox.zenodo.org/api/files/342b10af-1b07-4117-a022-79c43038e0dd',
           'discard': 'https://sandbox.zenodo.org/api/deposit/depositions/130529/actions/discard',
           'edit': 'https://sandbox.zenodo.org/api/deposit/depositions/130529/actions/edit',
           'files': 'https://sandbox.zenodo.org/api/deposit/depositions/130529/files',
           'html': 'https://sandbox.zenodo.org/deposit/130529',
           'latest_draft': 'https://sandbox.zenodo.org/api/deposit/depositions/130529',
           'latest_draft_html': 'https://sandbox.zenodo.org/deposit/130529',
           'newversion': 'https://sandbox.zenodo.org/api/deposit/depositions/130529/actions/newversion',
           'publish': 'https://sandbox.zenodo.org/api/deposit/depositions/130529/actions/publish',
           'registerc

In [71]:
#Put metadata
res = requests.put(
  f'{ZENODO_URL}/api/deposit/depositions/{newrecord_deposition_id}',
  params = params,
  headers = headers,
  data = json.dumps(zenodo_json))

assert (res.status_code == 200), f'Error updating metadata {newrecord_deposition_id}: {res.json()}'


In [72]:
#upload file
for file in zenodo_json['files']:
  filename = file['filename']
  with open(f'{publish_path}/{filename}', 'rb') as f:
    res = requests.put(
      f'{newrecord_bucket_url}/{filename}',
      params = params,
      data = f)
  assert (res.status_code == 201), f'Error uploading file {filename}: {res.json()}'
  assert (res.json()['size'] == file['size']), f'Error uploading file. Size mismatch: {res.json()['size']} != {file['size']}'
  assert (res.json()['checksum'] == file['checksum']), f'Error uploading file. Checksum mismatch: {res.json()['checksum']} != {file['checksum']}'
  #TODO: Verify MD5


In [33]:
res

<Response [201]>

In [73]:
res = requests.get(
  f'{ZENODO_URL}/api/deposit/depositions/{newrecord_deposition_id}',
  params = params,
  headers = headers
)
if res.status_code != 200:
  raise requests.RequestException(res.json())
res.json()

{'created': '2024-11-13T12:19:32.357191+00:00',
 'modified': '2024-11-13T12:19:38.436354+00:00',
 'id': 130529,
 'conceptrecid': '130528',
 'metadata': {'title': 'NetCDF Climate and Forecast (CF) Metadata Conventions',
  'publication_date': '2023-12-05',
  'description': 'This document describes the CF conventions for climate and forecast metadata designed to promote the processing and sharing of files created with the netCDF Application Programmer Interface. The conventions define metadata that provide a definitive description of what the data in each variable represents, and of the spatial and temporal properties of the data. This enables users of data from different sources to decide which quantities are comparable, and facilitates building applications with powerful extraction, regridding, and display capabilities. The CF conventions generalize and extend the COARDS conventions. The extensions include metadata that provides a precise definition of each variable via specification of

In [54]:
res = requests.get(
  f'{ZENODO_URL}/api/deposit/depositions',
  params = params,
  headers = headers
)
if res.status_code != 200:
  raise requests.RequestException(res.json())
res.json()

[{'created': '2024-11-12T14:07:59.435407+00:00',
  'modified': '2024-11-12T14:07:59.501685+00:00',
  'id': 129747,
  'conceptrecid': '129591',
  'conceptdoi': '10.5072/zenodo.129591',
  'metadata': {'title': 'NetCDF Climate and Forecast (CF) Metadata Conventions',
   'description': 'This document describes the CF conventions for climate and forecast metadata designed to promote the processing and sharing of files created with the netCDF Application Programmer Interface. The conventions define metadata that provide a definitive description of what the data in each variable represents, and of the spatial and temporal properties of the data. This enables users of data from different sources to decide which quantities are comparable, and facilitates building applications with powerful extraction, regridding, and display capabilities. The CF conventions generalize and extend the COARDS conventions. The extensions include metadata that provides a precise definition of each variable via speci

# "New" Version 1.0

In [74]:
publish_path='publish/1.0'
#Read metadata from zenodo.json file
with open(f'{publish_path}/zenodo.json', 'r') as f:
  zenodo_json = json.load(f)
zenodo_json

{'metadata': {'title': 'NetCDF Climate and Forecast (CF) Metadata Conventions',
  'description': 'This document describes the CF conventions for climate and forecast metadata designed to promote the processing and sharing of files created with the netCDF Application Programmer Interface. The conventions define metadata that provide a definitive description of what the data in each variable represents, and of the spatial and temporal properties of the data. This enables users of data from different sources to decide which quantities are comparable, and facilitates building applications with powerful extraction, regridding, and display capabilities. The CF conventions generalize and extend the COARDS conventions. The extensions include metadata that provides a precise definition of each variable via specification of a standard name, describes the vertical locations corresponding to dimensionless vertical coordinate values, and provides the spatial coordinates of non-rectilinear gridded d

In [75]:
#LATEST_DEPOSITION_ID = '129052'
LATEST_DEPOSITION_ID = '129592'
LATEST_DEPOSITION_ID = '130529'
# Create new version FROM record
#  f'{ZENODO_URL}/api/deposit/depositions/{LATEST_DEPOSITION_ID}/actions/newversion',
#  f'{ZENODO_URL}/api/records/{LATEST_DEPOSITION_ID}/versions',
res = requests.post(
  f'{ZENODO_URL}/api/deposit/depositions/{LATEST_DEPOSITION_ID}/actions/newversion',
  params = params,
  json = {},
  headers = headers
)
if res.status_code != 201:
  if res.status_code == 500 and res.json()['message'] == 'Misconfigured search.':
    warnings.warn(f'Error creating new version for deposition id {LATEST_DEPOSITION_ID}: {res.json()}')
  else:
    assert (res.status_code == 201), f'Error creating new version for deposition id {LATEST_DEPOSITION_ID}: {res.json()}'



In [76]:
res.json()

{'created': '2024-11-13T12:21:21.386728+00:00',
 'modified': '2024-11-13T12:21:21.534344+00:00',
 'id': 130532,
 'conceptrecid': '130528',
 'conceptdoi': '10.5072/zenodo.130528',
 'metadata': {'title': 'NetCDF Climate and Forecast (CF) Metadata Conventions',
  'description': 'This document describes the CF conventions for climate and forecast metadata designed to promote the processing and sharing of files created with the netCDF Application Programmer Interface. The conventions define metadata that provide a definitive description of what the data in each variable represents, and of the spatial and temporal properties of the data. This enables users of data from different sources to decide which quantities are comparable, and facilitates building applications with powerful extraction, regridding, and display capabilities. The CF conventions generalize and extend the COARDS conventions. The extensions include metadata that provides a precise definition of each variable via specificatio

In [ ]:
# TO BE KEEP in case temporla issues appear again
#TEMP_ID = 129747
#res = requests.get(
#  f'{ZENODO_URL}/api/deposit/depositions/{TEMP_ID}',
#  params = params,
#  headers = headers
#)
#assert (res.status_code == 200), f'Error retriving new verions deposition {TEMP_ID}: {res.json()}'

newversion_record = res.json()
newversion_deposition_id = newversion_record['id']
newversion_bucket_url = newversion_record['links']['bucket']
newversion_prereserve_doi = newversion_record['metadata']['prereserve_doi']['doi']
newversion_record

{'created': '2024-11-12T14:07:59.435407+00:00',
 'modified': '2024-11-12T14:22:52.429086+00:00',
 'id': 129747,
 'conceptrecid': '129591',
 'conceptdoi': '10.5072/zenodo.129591',
 'metadata': {'title': 'NetCDF Climate and Forecast (CF) Metadata Conventions',
  'description': 'This document describes the CF conventions for climate and forecast metadata designed to promote the processing and sharing of files created with the netCDF Application Programmer Interface. The conventions define metadata that provide a definitive description of what the data in each variable represents, and of the spatial and temporal properties of the data. This enables users of data from different sources to decide which quantities are comparable, and facilitates building applications with powerful extraction, regridding, and display capabilities. The CF conventions generalize and extend the COARDS conventions. The extensions include metadata that provides a precise definition of each variable via specificatio

In [50]:
res.json()

{'status': 500, 'message': 'Misconfigured search.'}

In [62]:
#delete files
for file in newversion_record['files']:
  res = requests.delete(
    f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}/files/{file['id']}',
    params = params,
  )
  assert (res.status_code == 204), f'Error deleting file {file['filename']}: {res.json()}'


In [65]:
#Put metadata
res = requests.put(
  f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}',
  params = params,
  headers = headers,
  data = json.dumps(zenodo_json))

assert (res.status_code == 200), f'Error updating metadata {newversion_deposition_id}: {res.json()}'


In [ ]:
#upload file
for file in zenodo_json['files']:
  filename = file['filename']
  with open(f'{publish_path}/{filename}', 'rb') as f:
    res = requests.put(
      f'{newversion_bucket_url}/{filename}',
      params = params,
      data = f)
  assert (res.status_code == 201), f'Error uploading file {filename}: {res.json()}'
  assert (res.json()['size'] == file['size']), f'Error uploading file. Size mismatch: {res.json()['size']} != {file['size']}'
  assert (res.json()['checksum'] == file['checksum']), f'Error uploading file. Checksum mismatch: {res.json()['checksum']} != {file['checksum']}'


# Parking Code

In [93]:
#Read metadata
with open('publish/1.11/metadata.json', 'r') as f:
  d = json.load(f)
json.dumps(d)
#Put metadata
r = requests.put(
  f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}',
  params = params,
  headers = headers,
  data = json.dumps(d))
r.status_code

200

In [94]:
#upload file
filename = 'cf-conventions.pdf'
with open(f'publish/1.11/{filename}', 'rb') as f:
  r = requests.put(
    f'{newversion_bucket_url}/{filename}',
    params = params,
    data = f)
  #TODO: Verify MD5
  if r.status_code != 201:
    raise Exception(r.json())

# New Version 1.12-rc7

In [97]:
LATEST_DEPOSITION_ID = '129052'
# Create new record
r = requests.post(f'{ZENODO_URL}/api/deposit/depositions/{LATEST_DEPOSITION_ID}/actions/newversion',
                   params=params,
                   json={},
                   headers=headers)
# 201
if r.status_code == 201:
  newversion_record = r.json()
  newversion_deposition_id = newversion_record['id']
  newversion_bucket_url = newversion_record['links']['bucket']
  newversion_prereserve_doi = newversion_record['metadata']['prereserve_doi']['doi']
else:
  raise requests.RequestException(r.json())



RequestException: {'status': 400, 'message': 'A validation error occurred.', 'errors': [{'field': 'files.enabled', 'messages': ['Please remove all files first.']}]}

In [98]:
#delete files
for file in newversion_record['files']:
  r = requests.delete(
    f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}/files/{file['id']}',
    params = params,
  )
  if r.status_code != 204:
    r.json()

In [104]:
#Read metadata
with open('publish/1.12-rc7/metadata.json', 'r') as f:
  d = json.load(f)
json.dumps(d)


'{"metadata": {"title": "NetCDF Climate and Forecast (CF) Metadata Conventions", "description": "This document describes the CF conventions for climate and forecast metadata designed to promote the processing and sharing of files created with the netCDF Application Programmer Interface. The conventions define metadata that provide a definitive description of what the data in each variable represents, and of the spatial and temporal properties of the data. This enables users of data from different sources to decide which quantities are comparable, and facilitates building applications with powerful extraction, regridding, and display capabilities. The CF conventions generalize and extend the COARDS conventions. The extensions include metadata that provides a precise definition of each variable via specification of a standard name, describes the vertical locations corresponding to dimensionless vertical coordinate values, and provides the spatial coordinates of non-rectilinear gridded da

In [105]:
#Put metadata
r = requests.put(
  f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}',
  params = params,
  headers = headers,
  data = json.dumps(d))
if r.status_code != 200:
    r.json()

In [106]:
#upload file
filename = 'cf-conventions.pdf'
with open(f'publish/1.12-rc7/{filename}', 'rb') as f:
  r = requests.put(
    f'{newversion_bucket_url}/{filename}',
    params = params,
    data = f)
  #TODO: Verify MD5
  if r.status_code != 201:
    raise Exception(r.json())

In [96]:
newversion_record

{'created': '2024-11-11T10:28:41.649677+00:00',
 'modified': '2024-11-11T10:28:41.800142+00:00',
 'id': 129255,
 'conceptrecid': '129051',
 'conceptdoi': '10.5072/zenodo.129051',
 'metadata': {'title': 'NetCDF Climate and Forecast (CF) Metadata Conventions',
  'description': 'This document describes the CF conventions for climate and forecast metadata designed to promote the processing and sharing of files created with the netCDF Application Programmer Interface. The conventions define metadata that provide a definitive description of what the data in each variable represents, and of the spatial and temporal properties of the data. This enables users of data from different sources to decide which quantities are comparable, and facilitates building applications with powerful extraction, regridding, and display capabilities. The CF conventions generalize and extend the COARDS conventions. The extensions include metadata that provides a precise definition of each variable via specificatio